In [2]:
from __future__ import division
from time import time
import numpy as np
np.random.seed(1337)
from scipy.stats import poisson
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.datasets import load_digits

In [ ]:
alpha = 6
sigma = 5
sigma_a = 5

In [ ]:
def log_X_given_Z(X, Z):
    N = Z.shape[0]
    K = Z.shape[1]
    D = X.shape[1]
    I1 = np.identity(K)
    I2 = np.identity(N)
    
    rep_term = Z.T @ Z + (sigma ** 2 / sigma_a ** 2) * I1
    # get log of numerator and denominator, and return log prob
    num = (-1/(2*(sigma**2))) * np.trace(X.T @ (I2 - Z @ np.linalg.inv(rep_term) @ Z.T) @ X)
    denom = (np.log(2 * np.pi) * (N * D / 2)) + (np.log(sigma) * ((N - K) * D)) + (np.log(sigma_a) * (K * D)) + \
            (np.log(np.linalg.det(rep_term)) * (D / 2))
          
    return (num - denom)

def log_X_given_Z_A(X, Z, A):
    N = Z.shape[0]
    K = Z.shape[1]
    D = X.shape[1]

    # return log prob
    return -np.log(2*np.pi*sigma**2) * (N * D / 2) - (1/(2*sigma**2)) * np.trace((X - Z @ A).T @ (X - Z @ A))

def resample(X, Z):
    N = Z.shape[0]
    K = Z.shape[1]
    D = X.shape[1]
    I = np.identity(K)
    
    rep_term = np.linalg.inv(Z.T @ Z + ((sigma/sigma_a)**2) * I)
    means = rep_term @ Z.T @ X
    cov = (sigma**2) * rep_term
                 
    return np.array([np.random.multivariate_normal(mean, cov) for mean in means.T]).T

def make_additional_columns(N, row, num_cols):
    add_cols = np.zeros((N, num_cols))
    add_cols[row] = 1
    return add_cols

def generate_sample(Z_row):
    '''
    Generate random output sample given feature matrix. 
    '''
    K = len(Z_row)
    samples = []
    probs = []
    for _ in range(1000):
        sample = np.random.uniform(size = K)
        prob = log_X_given_Z(sample, Z_row)
        samples.append(sample)
        probs.append(prob)
    generated_sample = np.random.choice(samples, p = probs)
    
    return generated_sample

def sample_row(X, Z, row):
    '''
    Sample each row using collapsed Gibbs sampler. 
    '''
    N = Z.shape[0]
    K = Z.shape[1]
    D = X.shape[1]
    truncation = D // 2
    Z_new = np.copy(Z)
    
    # sample existing features
    for i in range(K):
        col_sum = Z[:, i].sum()
        Z_new[row, i] = 0
        log_prob_0 = log_X_given_Z(X, Z_new) 
        Z_new[row, i] = 1
        log_prob_1 = log_X_given_Z(X, Z_new) 
        M = max(log_prob_0, log_prob_1)
        prob = (((N - col_sum) / N) * np.exp(log_prob_0 - M)) / (((N - col_sum) / N) * np.exp(log_prob_0 - M) + \
                                                           (col_sum / N) * np.exp(log_prob_1 - M))
        Z_new[row, i] = np.random.choice([0, 1], p = [prob, 1 - prob])
        
    # sample new features for Z and A
    probs = []
    for i in range(truncation + 1):
        Z1 = np.concatenate((Z_new, make_additional_columns(N, row, i)), axis = 1)
        probs.append(log_X_given_Z(X, Z1))
        
    # prior probabilities
    prior_probs = np.copy(probs)
    prior_probs = prior_probs - np.max(prior_probs)
    prior_probs = np.exp(prior_probs)
    
    probs = np.array(probs) - np.max(probs)
    probs = np.exp(probs)
    probs = probs * np.array([poisson.pmf(i, alpha / N) for i in range(truncation + 1)])
    probs /= np.sum(probs)
    new_features = np.random.choice(np.arange(truncation + 1), p = probs)
    Z_new = np.concatenate((Z_new, make_additional_columns(N, row, new_features)), axis = 1)
    
    return Z_new, prior_probs, probs

In [28]:
digits = load_digits().data # digits are 8x8
for i in range(digits.shape[0]):
    digits[i] = digits[i] / digits[i].max()
im_dim = 8
dist_matrix = np.zeros((64, 64))
for i in range(64):
    for j in range(64):
        row1 = i // 8
        col1 = i % 8
        row2 = j // 8
        col2 = j % 8
        dist_matrix[i][j] = np.sqrt((row1 - row2) ** 2 + (col1 - col2) ** 2)

def wasserstein(p_, q_, cost_matrix = dist_matrix, epsilon = 0.05 * np.median(dist_matrix), niterations = 100):
    # in case p_ or q_ contain 0's
    p = p_ + 1e-10
    q = q_ + 1e-10
    N = len(p)
    M = len(q)
    K = np.exp((-1 / epsilon) * cost_matrix)
    K_transpose = K.transpose()
    K_tilde = np.diag(1 / p) @ K
    u = np.repeat(1. / N, N)
    for _ in range(niterations):
        u = 1 / (K_tilde @ (q / (K_transpose @ u)))
    v = q / (K_transpose @ u)
    transportmatrix = np.diag(u) @ K @ np.diag(v)
    uXIv = u * ((K * cost_matrix) @ v)
    d = uXIv.sum()
    return d, transportmatrix, u, v

In [ ]:
def ABC_Gibbs(X, A, Z, epsilon = 0.5):
    # right now we use the collapsed Gibbs sampler
    N = Z.shape[0]
    K = Z.shape[1]
    D = X.shape[1]
    Z_new = Z.copy()
    for i in range(N):
        while True:
            Z_sample, _, _ = sample_row(X, Z_new, i)
            Z_row = Z_sample[i]
            sample = generate_sample(Z_row)
            if wasserstein(sample, X[i]) < epsilon:
                Z_new = Z_sample
                break